In [8]:
import os.path
import random
import time
from functools import partial
from tkinter import *
import sys

from lib.EightPuzzle import EightPuzzle

from queue import PriorityQueue

In [9]:
root = Tk()

# # Prompt for goal state
# goal_state = input("Enter the goal state (comma-separated values): ")
# goal_state = list(map(int, goal_state.split(',')))

# # Prompt for start state
# start_state = input("Enter the start state (comma-separated values): ")
# start_state = list(map(int, start_state.split(',')))

state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
puzzle = EightPuzzle(tuple(state))
solution = None

b = [None] * 9

## Helper Functions & Classes

In [10]:
def get_cost(action,cost_uptill_now=0):
    cost = 0
    if action == "UP":
        cost = 2
    elif action == "LEFT":
        cost = 1
    elif action == "RIGHT":
        cost = 4
    elif action == "DOWN":
        cost = 3
    else:
        cost = 0
    return cost + cost_uptill_now


def get_heuristic_h1(state, goal):
    # Manhattan distance  
    distance = 0
    state = list(state)
    goal = list(goal)
    for i in range(3):
        for j in range(3):
            value = state[i * 3 + j]
            if value != 0:
                goal_index = goal.index(value)
                goal_row = goal_index // 3
                goal_col = goal_index % 3
                distance += abs(i - goal_row) + abs(j - goal_col)
    return distance

def get_heuristic_h2(state, goal):
    # number of misplaced tiles 
    misplaced_tiles = 0
    state = list(state)
    goal = list(goal)
    for i in range(len(state)):
        if state[i] != 0 and state[i] != goal[i]:
            misplaced_tiles += 1

    return misplaced_tiles

def get_heuristic_max(state,goal):
    return max(get_heuristic_h1(state,goal),get_heuristic_h2(state,goal))

class Node:
    def __init__(self, state, parent=None, action=None, cost=0, heuristic=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = cost
        self.heuristic = heuristic
        self.priority = cost + heuristic
    
    def __lt__(self, other):
        return self.priority < other.priority

## Greedy BFS

In [11]:
def BestFS(p):
    expanded_node=0
    fringe_node=0
    path_length=0
    frontier = PriorityQueue()
    frontier.put(Node(p, None, None, 0, get_heuristic_h2(p,puzzle.goal)))
    
    explored = set()
    
    while not frontier.empty():
        current_node = frontier.get()
        expanded_node+=1
        current_state = current_node.state
        
        if puzzle.goal_test(current_state):# backtracking using parent in node class
            solution = []
            while current_node.parent is not None:
                solution.append(current_node.action)
                current_node = current_node.parent
            path_length+=len(solution)
            solution.reverse()
            return solution # Reverse the path 
        
        explored.add(tuple(current_state))
        
        for action in puzzle.actions(current_state):
            next_state = puzzle.result(current_state, action)
            if tuple(next_state) not in explored:
                heuristic = get_heuristic_h2(next_state,puzzle.goal)
                frontier.put(Node(next_state, current_node, action, 0, heuristic))
            fringe_node=max(fringe_node,frontier.qsize())
    
    return []

# AStar

In [12]:
def astar(p):
    expanded_node=0
    fringe_node=0
    path_length=0
    frontier = PriorityQueue()
    frontier.put(Node(p, None, None, 0, get_heuristic_h1(p,puzzle.goal)))
    
    explored = set()
    
    while not frontier.empty():
        current_node = frontier.get()
        expanded_node+=1
        current_state = current_node.state
        
        if puzzle.goal_test(current_state):# backtracking using parent in node class
            solution = []
            while current_node.parent is not None:
                solution.append(current_node.action)
                current_node = current_node.parent
            solution.reverse()
            path_length+=len(solution)
            return solution
        
        explored.add(tuple(current_state))
        
        for action in puzzle.actions(current_state):
            next_state = puzzle.result(current_state, action)
            if tuple(next_state) not in explored:
                cost = get_cost(action, current_node.cost)
                heuristic = get_heuristic_h1(next_state,puzzle.goal)
                frontier.put(Node(next_state, current_node, action, cost, heuristic))
            fringe_node=max(fringe_node,frontier.qsize())
    
    return []

In [13]:
def solve():
    """
    Solves the puzzle using astar_search using puzzle array
    p contains the input puzzle
    goal
    
    return an array of this structure
    ['UP', 'LEFT', 'UP', 'RIGHT', 'DOWN', 'LEFT', 'LEFT', 'DOWN', 'RIGHT', 'DOWN']
    """
    global puzzle
    p = puzzle.initial
    

    #return astar(p)
    return astar(p)

In [14]:


def scramble():
    """Scrambles the puzzle starting from the goal state"""

    global state
    global puzzle
    possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    scramble = []
    for _ in range(60):
        scramble.append(random.choice(possible_actions))

    for move in scramble:
        if move in puzzle.actions(state):
            state = list(puzzle.result(state, move))
            puzzle = EightPuzzle(tuple(state))
            create_buttons()


def solve_steps():
    """Solves the puzzle step by step"""

    global puzzle
    global solution
    global state
    solution = solve()
    print(solution)

    for move in solution:
        state = puzzle.result(state, move)
        create_buttons()
        root.update()
        root.after(1, time.sleep(0.75))


def exchange(index):
    """Interchanges the position of the selected tile with the zero tile under certain conditions"""

    global state
    global solution
    global puzzle
    zero_ix = list(state).index(0)
    actions = puzzle.actions(state)
    current_action = ''
    i_diff = index // 3 - zero_ix // 3
    j_diff = index % 3 - zero_ix % 3
    if i_diff == 1:
        current_action += 'DOWN'
    elif i_diff == -1:
        current_action += 'UP'

    if j_diff == 1:
        current_action += 'RIGHT'
    elif j_diff == -1:
        current_action += 'LEFT'

    if abs(i_diff) + abs(j_diff) != 1:
        current_action = ''

    if current_action in actions:
        b[zero_ix].grid_forget()
        b[zero_ix] = Button(root, text=f'{state[index]}', width=6, font=('Helvetica', 40, 'bold'),
                            command=partial(exchange, zero_ix))
        b[zero_ix].grid(row=zero_ix // 3, column=zero_ix % 3, ipady=40)
        b[index].grid_forget()
        b[index] = Button(root, text=None, width=6, font=('Helvetica', 40, 'bold'), command=partial(exchange, index))
        b[index].grid(row=index // 3, column=index % 3, ipady=40)
        state[zero_ix], state[index] = state[index], state[zero_ix]
        puzzle = EightPuzzle(tuple(state))


def create_buttons():
    """Creates dynamic buttons"""

    # TODO: Find a way to use grid_forget() with a for loop for initialization
    b[0] = Button(root, text=f'{state[0]}' if state[0] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 0))
    b[0].grid(row=0, column=0, ipady=40)
    b[1] = Button(root, text=f'{state[1]}' if state[1] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 1))
    b[1].grid(row=0, column=1, ipady=40)
    b[2] = Button(root, text=f'{state[2]}' if state[2] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 2))
    b[2].grid(row=0, column=2, ipady=40)
    b[3] = Button(root, text=f'{state[3]}' if state[3] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 3))
    b[3].grid(row=1, column=0, ipady=40)
    b[4] = Button(root, text=f'{state[4]}' if state[4] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 4))
    b[4].grid(row=1, column=1, ipady=40)
    b[5] = Button(root, text=f'{state[5]}' if state[5] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 5))
    b[5].grid(row=1, column=2, ipady=40)
    b[6] = Button(root, text=f'{state[6]}' if state[6] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 6))
    b[6].grid(row=2, column=0, ipady=40)
    b[7] = Button(root, text=f'{state[7]}' if state[7] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 7))
    b[7].grid(row=2, column=1, ipady=40)
    b[8] = Button(root, text=f'{state[8]}' if state[8] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 8))
    b[8].grid(row=2, column=2, ipady=40)


def create_static_buttons():
    """Creates scramble and solve buttons"""

    scramble_btn = Button(root, text='Scramble', font=('Helvetica', 30, 'bold'), width=8, command=partial(init))
    scramble_btn.grid(row=3, column=0, ipady=10)
    solve_btn = Button(root, text='Solve', font=('Helvetica', 30, 'bold'), width=8, command=partial(solve_steps))
    solve_btn.grid(row=3, column=2, ipady=10)


def init():
    """Calls necessary functions"""

    global state
    global solution
    

    state = [1, 2, 3, 4, 5, 6, 7, 8,0,]
    scramble()
    create_buttons()
    create_static_buttons()


init()
root.mainloop()

['DOWN', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'DOWN', 'DOWN', 'LEFT', 'UP', 'UP', 'RIGHT', 'DOWN', 'DOWN', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'DOWN']
